In [1]:
% % capture - -no-stderr
%pip install - -upgrade - -quiet langchain langchain-community beautifulsoup4
%pip install - qU langchain_chroma

# Local inference and embeddings via Ollama
%pip install - qU langchain_ollama


In [4]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()


In [2]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.2")

llm.invoke("The first man on the moon was ...")


/Users/kjevaji/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


'...Neil Armstrong. He stepped out of the lunar module Eagle and became the first person to set foot on the Moon on July 20, 1969, during the Apollo 11 mission. His famous words upon exiting the spacecraft were: "That\'s one small step for man, one giant leap for mankind."'

In [3]:
import bs4
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings


# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://blog.logrocket.com/ux-design/gestalt-laws-of-grouping/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header", "lr-content")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(
    documents=all_splits, embedding=local_embeddings)
retriever = vectorstore.as_retriever()


# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


In [5]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(
    history_aware_retriever, question_answer_chain)


In [6]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "Summarize all information"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are three questions I can ask?"
ai_msg_2 = rag_chain.invoke(
    {"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])


I don't know the answer to your question about what questions to ask. However, I can provide information on the Gestalt laws of grouping, which include:

1. Law of Closure: Our brains fill gaps and recognize complete objects even if parts are missing.
2. Law of Similarity: Visual entities with similar characteristics (size, shape, color) are perceived as groups.

These laws help designers group UI elements effectively.


In [7]:
print(chat_history)


[HumanMessage(content='Summarize all information', additional_kwargs={}, response_metadata={}), AIMessage(content='The context discusses various principles in UI/UX design related to grouping elements. The laws include:\n\n1. Law of Closure: Our brains fill gaps and recognize complete objects even if parts are missing.\n2. Law of Similarity: Visual entities with similar characteristics (size, shape, color) are perceived as groups.\n\nThese laws help designers group UI elements effectively, such as using proximity, connectedness, similarity, and continuity to create cohesive designs that improve user interaction and experience.', additional_kwargs={}, response_metadata={})]


In [8]:
ai_msgs = []
ai_msgs.append(rag_chain.invoke({
    "input": "Tell me more",
    "chat_history": chat_history
})
)


In [14]:
print(ai_msgs[-1]["answer"])

The Gestalt laws used in grouping UI/UX design elements include the Law of Proximity (entities close together are perceived as a group), the Law of Connectedness (visual connections between elements create groups), and the Law of Similarity (elements with similar characteristics are perceived as groups). These principles guide designers to effectively group UI elements, such as using color, size, shape, and proximity. By applying these laws, designers can create cohesive designs that improve user interaction and experience.


In [15]:
print(docs)

[Document(metadata={'source': 'https://blog.logrocket.com/ux-design/gestalt-laws-of-grouping/', 'language': 'No language found.'}, page_content="\n\n\n\nIn UI/UX design, grouping is a way to arrange multiple visual elements based on visual characteristics to reduce user cognitive load. So, UI/UX designers typically group similar UI elements or element collections based on various design principles to offer users productive, friendly, intuitive human-computer interaction.\nAnd to create effective interface designs, UI/UX designers should use effective, optimal principles for grouping similar elements or segments based on proven human psychological facts.\nGestalt psychology, founded by Max Wertheimer and two other German psychologists, introduced a set of grouping principles. These laws are based on a study that explores how the human brain processes visual patterns and complex visual entities. Almost all modern software UIs optimally adhere to these Gestalt grouping laws for creating i